In [19]:
import pandas as pd

In [28]:
columns_to_import = ["Case Type","Region","Case Number","Case Name","Status","Date Filed","Date Closed","Reason Closed"
                     ,"City","States & Territories","Employees on charge/petition","Allegations","Participants","Union",
                     "Unit Sought","Voters"]
column_dtypes = {c:"string" for c in columns_to_import}

In [29]:
df1 = pd.read_csv("1978-1999.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"])
df2 = pd.read_csv("2000-2004.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"])
df3 = pd.read_csv("2005-2009.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"])
df4 = pd.read_csv("2015-2019.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"])

0   1999-12-30
1   1999-12-30
2   1999-12-30
3   1999-12-30
4   1999-12-30
Name: Date Filed, dtype: datetime64[ns]

In [10]:
df4["Date Filed"].tail()

97453    01/02/2015
97454    01/02/2015
97455    01/02/2015
97456    01/02/2015
97457    01/02/2015
Name: Date Filed, dtype: object